1. Importing Liabriaries and Supress Warnings

In [ ]:
import pandas as pd
import numpy as np

1.2 Importing Graphs

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
import missingno as msno
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression,LogisticRegression, Lasso, Ridge
from sklearn import metrics
from sklearn.manifold import TSNE
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import IncrementalPCA
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score, precision_score, recall_score, precision_recall_curve, f1_score, accuracy_score, r2_score, mean_squared_error

1.4 Importing Warnings

In [ ]:
import warnings
warnings.filterwarnings('ignore')

1.5 Increasing Row and Column Size

In [ ]:
pd.set_option('display.max_rows', 3000)
pd.set_option('display.max_columns',3000)

2. Loading Data

2.1 Importing CSV Data from file

In [ ]:
news_data = pd.read_csv('/content/drive/MyDrive/Upgrad/News Popularity Prediction/train.csv')
news_unseen=pd.read_csv("/content/drive/MyDrive/Upgrad/News Popularity Prediction/test.csv")
data_dict=pd.read_csv("/content/drive/MyDrive/Upgrad/News Popularity Prediction/data_dictionary.csv")

In [ ]:
news_unseen.head()

### 3. Understanding Data 

In [ ]:
print(news_data.shape)
print(news_unseen.shape)
print(data_dict.shape)

In [ ]:
data_dict

In [ ]:
news_data.head()

In [ ]:
news_data.dtypes

### 4. Data Prepocessing

In [ ]:
news_data.head()

In [ ]:
news_data.info(verbose=True)

In [ ]:
news_data.describe(include='all')

In [ ]:
# Checking the Churn Rate
shares_percent = (sum(news_data['shares'])/len(news_data['shares'].index))*100
shares_percent

In [ ]:
news_data = news_data.drop('url',axis=1)

In [ ]:
news_data.shape

4.2 Create X, y and then Train test split

In [ ]:
X = news_data.set_index('id')
y = news_data['shares']

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
y_train.head()

In [ ]:
X_train.head()

#### 4.3 Handling Missing data

##### 4.3.1 Checking Missing Values

In [ ]:
# Checking percentage of missing values before removing the missing values
round(100*(X_train.isnull().sum()/len(X_train.index)), 2)

##### 4.3.2 Imputing Missing Values

In [ ]:
media_features_missing_values=['num_imgs','num_videos']
reference_features_missing_values =['self_reference_min_shares','self_reference_max_shares','self_reference_avg_sharess','num_hrefs','num_self_hrefs']
weekday_missing_values=['is_weekend','weekday_is_monday','weekday_is_tuesday','weekday_is_wednesday','weekday_is_thursday','weekday_is_friday','weekday_is_saturday','weekday_is_sunday']
subjectivity_features_missing_values = ['global_subjectivity', 'title_subjectivity', 'abs_title_subjectivity']
keyword_features_missing_values=['kw_min_min','kw_min_max','kw_min_avg','kw_max_min','kw_max_max','kw_max_avg','kw_avg_min','kw_avg_max','kw_avg_avg','num_keywords']

sentiment_features_missing_values = ['global_sentiment_polarity', 'global_rate_positive_words','global_rate_negative_words', 'rate_positive_words','rate_negative_words', 'avg_positive_polarity',
                     'min_positive_polarity', 'max_positive_polarity','avg_negative_polarity', 'min_negative_polarity','max_negative_polarity', 'title_sentiment_polarity',
                       'abs_title_sentiment_polarity']


In [ ]:
median_imputation = SimpleImputer(missing_values=np.nan, strategy='median')

In [ ]:
X_train[media_features_missing_values]=median_imputation.fit_transform(X_train[media_features_missing_values])
X_train[reference_features_missing_values]=median_imputation.fit_transform(X_train[reference_features_missing_values])
X_train[weekday_missing_values]=median_imputation.fit_transform(X_train[weekday_missing_values])
X_train[subjectivity_features_missing_values]=median_imputation.fit_transform(X_train[subjectivity_features_missing_values])
X_train[keyword_features_missing_values]=median_imputation.fit_transform(X_train[keyword_features_missing_values])
X_train[sentiment_features_missing_values]=median_imputation.fit_transform(X_train[sentiment_features_missing_values])


In [ ]:
X_test[media_features_missing_values]=median_imputation.fit_transform(X_test[media_features_missing_values])
X_test[reference_features_missing_values]=median_imputation.fit_transform(X_test[reference_features_missing_values])
X_test[weekday_missing_values]=median_imputation.fit_transform(X_test[weekday_missing_values])
X_test[subjectivity_features_missing_values]=median_imputation.fit_transform(X_test[subjectivity_features_missing_values])
X_test[keyword_features_missing_values]=median_imputation.fit_transform(X_test[keyword_features_missing_values])
X_test[sentiment_features_missing_values]=median_imputation.fit_transform(X_test[sentiment_features_missing_values])

In [ ]:
X_train.isnull().sum()

In [ ]:
X_test.isnull().sum()

In [ ]:
msno.bar(X_train)

In [ ]:
X_train.describe()

In [ ]:
X_train.head()

#### 5. Outlier Treatment

##### 5.1.1 Checking Outliers on Train Set

In [ ]:
plt.figure(figsize=(15,8))
plt.xticks(rotation=45)
sns.boxplot(data = X_train)
plt.show()

##### 5.1.2 Checking Outliers on Test Set

In [ ]:
plt.figure(figsize=(15,8))
plt.xticks(rotation=45)
sns.boxplot(data = X_test)
plt.show()

##### 5.2 Removing Outliers

##### 5.2.1 Removing Outliers on Train Set

In [ ]:
def cap_outliers(array, k=3):
    upper_limit = array.mean() + k*array.std()
    lower_limit = array.mean() - k*array.std()
    array[array<lower_limit] = lower_limit
    array[array>upper_limit] = upper_limit
    return array

In [ ]:
X_train = X_train.apply(cap_outliers, axis=0)

plt.figure(figsize=(15,8))
plt.xticks(rotation=45)
sns.boxplot(data = X_train)

##### 5.2.2 Removing Outliers on Test Set

In [ ]:
def cap_outliers(array, k=3):
    upper_limit = array.mean() + k*array.std()
    lower_limit = array.mean() - k*array.std()
    array[array<lower_limit] = lower_limit
    array[array>upper_limit] = upper_limit
    return array

In [ ]:
X_test = X_test.apply(cap_outliers, axis=0)

plt.figure(figsize=(15,8))
plt.xticks(rotation=45)
sns.boxplot(data = X_test)

### 8.Feature Engineering And Selection

### 8.1.1 Scaling on Train Set

In [ ]:
X_train.describe()

In [ ]:
scale = StandardScaler()
X_train[:] = scale.fit_transform(X_train)

In [ ]:
X_train.describe()

### 8.1.2 Scaling on Test Set

In [ ]:
X_test.describe()

In [ ]:
scale = StandardScaler()
X_test[:] = scale.fit_transform(X_test)

In [ ]:
X_test.describe()

In [ ]:
X_train.shape

In [ ]:
X_test.shape

### 7. Exploratory Data Analysis

#### 7.1 Plotting Heatmap

In [ ]:
plt.figure(figsize=(25,25))
sns.heatmap(X_train.corr(),annot=True)

### 8.Feature Engineering And Selection

#### 8.1 Performing PCA Train Data

In [ ]:
pca = PCA(random_state=42)

In [ ]:
pca.fit(X_train)

In [ ]:
pca.components_

#### 8.2 Components From PCA

In [ ]:
pca.explained_variance_ratio_

#### 8.3 Making Scree Plot For Explained Variance

In [ ]:
var_cumu = np.cumsum(pca.explained_variance_ratio_)

In [ ]:
fig = plt.figure(figsize=[12,8])
plt.vlines(x=15, ymax=1, ymin=0, colors="r", linestyles="--")
plt.hlines(y=0.95, xmax=50, xmin=0, colors="g", linestyles="--")
plt.plot(var_cumu)
plt.ylabel("Cumulative variance explained")
plt.show()


#### 8.4 Perform PCA With 35 Components

In [ ]:
pca_final = IncrementalPCA(n_components=35)

In [ ]:
X_train_pca = pca_final.fit_transform(X_train)

In [ ]:
X_train_pca.shape

In [ ]:
corrmat = np.corrcoef(X_train_pca.transpose())

In [ ]:
corrmat.shape

#### 8.5 Plotting Heatmap For Corr Matrix

In [ ]:
plt.figure(figsize=[20,20])
sns.heatmap(corrmat, annot=True)

In [ ]:
X_test_pca = pca_final.transform(X_test)
X_test_pca.shape

### 9. Model Building

#### 9.1 Linear Regression on Seen Data

##### 9.1.1 Train Set

In [ ]:
reg = LinearRegression() 
reg.fit(X_train,y_train)

In [ ]:
# Predictions on the basis of the model
y_pred = reg.predict(X_train)
y_pred

In [ ]:
r2_score(y_train, y_pred) 

In [ ]:
#Residual Sum of Squares = Mean_Squared_Error * Total number of datapoints
rss = np.sum(np.square(y_train - y_pred))
print(rss)
mse = mean_squared_error(y_train, y_pred)
print(mse)
# Root Mean Squared Error
rmse = mse**0.5
print(rmse)

##### 9.1.2 Test Set

In [ ]:
reg = LinearRegression() 
reg.fit(X_test,y_test)

In [ ]:
# Predictions on the basis of the model
y_pred = reg.predict(X_test)
y_pred

In [ ]:
r2_score(y_test, y_pred)

In [ ]:
#Residual Sum of Squares = Mean_Squared_Error * Total number of datapoints
rss = np.sum(np.square(y_test - y_pred))
print(rss)
mse = mean_squared_error(y_test, y_pred)
print(mse)
# Root Mean Squared Error
rmse = mse**0.5
print(rmse)

#### 9.1 Linear Regression on Unseen Data

In [ ]:
news_unseen.head()

In [ ]:
submission_data = news_unseen.drop('url',axis=1)

In [ ]:
submission_data.set_index('id')

In [ ]:
# Checking percentage of missing values before removing the missing values
round(100*(submission_data.isnull().sum()/len(submission_data.index)), 2)

In [ ]:
media_features_missing_values=['num_imgs','num_videos']
reference_features_missing_values =['self_reference_min_shares','self_reference_max_shares','self_reference_avg_sharess','num_hrefs','num_self_hrefs']
weekday_missing_values=['is_weekend','weekday_is_monday','weekday_is_tuesday','weekday_is_wednesday','weekday_is_thursday','weekday_is_friday','weekday_is_saturday','weekday_is_sunday']
subjectivity_features_missing_values = ['global_subjectivity', 'title_subjectivity', 'abs_title_subjectivity']
keyword_features_missing_values=['kw_min_min','kw_min_max','kw_min_avg','kw_max_min','kw_max_max','kw_max_avg','kw_avg_min','kw_avg_max','kw_avg_avg','num_keywords']

sentiment_features_missing_values = ['global_sentiment_polarity', 'global_rate_positive_words','global_rate_negative_words', 'rate_positive_words','rate_negative_words', 'avg_positive_polarity',
                     'min_positive_polarity', 'max_positive_polarity','avg_negative_polarity', 'min_negative_polarity','max_negative_polarity', 'title_sentiment_polarity',
                       'abs_title_sentiment_polarity']

In [ ]:
median_imputation = SimpleImputer(missing_values=np.nan, strategy='median')

In [ ]:
submission_data[media_features_missing_values]=median_imputation.fit_transform(submission_data[media_features_missing_values])
submission_data[reference_features_missing_values]=median_imputation.fit_transform(submission_data[reference_features_missing_values])
submission_data[weekday_missing_values]=median_imputation.fit_transform(submission_data[weekday_missing_values])
submission_data[subjectivity_features_missing_values]=median_imputation.fit_transform(submission_data[subjectivity_features_missing_values])
submission_data[keyword_features_missing_values]=median_imputation.fit_transform(submission_data[keyword_features_missing_values])
submission_data[sentiment_features_missing_values]=median_imputation.fit_transform(submission_data[sentiment_features_missing_values])

In [ ]:
# Checking percentage of missing values after removing the missing values
round(100*(submission_data.isnull().sum()/len(submission_data.index)), 2)

In [ ]:
def cap_outliers(array, k=3):
    upper_limit = array.mean() + k*array.std()
    lower_limit = array.mean() - k*array.std()
    array[array<lower_limit] = lower_limit
    array[array>upper_limit] = upper_limit
    return array

In [ ]:
submission_data = submission_data.apply(cap_outliers, axis=0)

In [ ]:
scale = StandardScaler()
submission_data[:] = scale.fit_transform(submission_data)

In [ ]:
prob_news_popularity_test = reg.predict(submission_data)
prob_news_popularity_test

In [ ]:
prob_news_popularity_test_df = pd.DataFrame(prob_news_popularity_test)
# Converting to column dataframe
prob_news_popularity_test_final = prob_news_popularity_test_df.iloc[:,[0]]
prob_news_popularity_test_final.head()

In [ ]:
submission_data.head()

In [ ]:
submission_data.reset_index(inplace=True)

In [ ]:
submission_data

In [ ]:
submission_data=pd.concat([submission_data,prob_news_popularity_test_df],axis=1)
submission_data

In [ ]:
submission_data.rename(columns={0:'shares_no'},inplace=True)

In [ ]:
submission_data.head()

In [ ]:
submission_data['shares']=submission_data.shares_no.map( lambda x: 1 if x > 0.5 else 0)
output = submission_data[['id','shares']]
output.head()

#### 9.2 Polynomial Regression on Seen Data

##### 9.2.1 On Train Set

In [ ]:
poly = PolynomialFeatures(degree=2)
poly_features = poly.fit_transform(X_train_pca)
poly_features.shape

In [ ]:
polyreg = LinearRegression()
polyreg.fit(poly_features,y_train)

In [ ]:
y_pred2 = polyreg.predict(poly.fit_transform(X_train_pca)) # store predictions from the polynomial regression in the variable y_pred5
print(r2_score(y_train, y_pred2))

In [ ]:
# Metrics to assess model performance
rss = np.sum(np.square(y_train - y_pred2))
print(rss)
mse = mean_squared_error(y_train, y_pred2)
print(mse)
rmse = mse**0.5
# Root Mean Squared Error
print(rmse)

##### 9.2.1 On Test Set

In [ ]:
poly = PolynomialFeatures(degree=2)
poly_features = poly.fit_transform(X_test_pca)
poly_features.shape

In [ ]:
polyreg1 = LinearRegression()
polyreg1.fit(poly_features,y_test)

In [ ]:
y_pred3 = polyreg1.predict(poly.fit_transform(X_test_pca)) # store predictions from the polynomial regression in the variable y_pred5
print(r2_score(y_test, y_pred3))

In [ ]:
# Metrics to assess model performance
rss = np.sum(np.square(y_test - y_pred3))
print(rss)
mse = mean_squared_error(y_test, y_pred3)
print(mse)
rmse = mse**0.5
# Root Mean Squared Error
print(rmse)

#### 9.2 Polynomial Regression on UnSeen Data

In [ ]:
poly_test = PolynomialFeatures(degree=2)
poly_features = poly_test.fit_transform(submission_data)
poly_features.shape

In [ ]:
polyreg_test = LinearRegression()
polyreg_test.fit(poly_features,submission_data)

In [ ]:
y_pred_test = polyreg_test.predict(poly.fit_transform(submission_data))

In [ ]:
submission_data.reset_index(inplace=True)
output=submission_data[['id','shares']]
output.head()

#### 9.3 Ridge Regression on Seen Data

##### 9.3.1 On Train Set

In [ ]:
# Applying Ridge Regression with varying the hyperparameter 'lambda' on Train Set

X_seq = np.linspace(X_train.min(),X_train.max(),300).reshape(-1,1) 
lambdas = [0, 0.001, 0.01, 0.1, 1, 10, 100, 1000] # Higher the value of lambda, 
                                                  # more the regularization
for i in lambdas: # for each lambda we get different model coefficients
    degree = 2 # Degree for polynomial regression - chose 5 since this is the lowest number that gave a perfect fit
    # Creating degree 5 features
    ridgecoef = PolynomialFeatures(degree)
    # Transforming input features to polynomial features (1, x1, x2)    
    X_poly = ridgecoef.fit_transform(X_train)
    ridgereg = Ridge(alpha = i) # Initialize the Ridge Regression model with a specific lambda
    ridgereg.fit(X_poly, y_train) # fit the model on the polynomial features
    
  
    
    #Computing the rss,mse,rmse,r2 score
    y_pred = ridgereg.predict(ridgecoef.fit_transform(X_train))
    rss = np.sum(np.square(y_train - y_pred))
    mse = mean_squared_error(y_train, y_pred)
    rmse = mse**0.5

    print("Polynomial regression with degree "+str(degree) + " and lambda = " + str(i))
    print("r2 score = " + str(r2_score(y_train, y_pred))) 
    print("rss = " + str(rss))
    print("mse = " + str(rmse))
    print("rmse = " + str(rmse))
    print(ridgereg.coef_) # model coefficients

##### 9.3.2 On Test Set

In [ ]:
# Applying Ridge Regression with varying the hyperparameter 'lambda' on Test Set

X_seq = np.linspace(X_test.min(),X_test.max(),300).reshape(-1,1) 
lambdas = [0, 0.001, 0.01, 0.1, 1, 10, 100, 1000] # Higher the value of lambda, 
                                                  # more the regularization
for i in lambdas: # for each lambda we get different model coefficients
    degree = 2 # Degree for polynomial regression - chose 5 since this is the lowest number that gave a perfect fit
    # Creating degree 5 features
    ridgecoef = PolynomialFeatures(degree)
    # Transforming input features to polynomial features (1, x1, x2)    
    X_poly = ridgecoef.fit_transform(X_test)
    ridgereg = Ridge(alpha = i) # Initialize the Ridge Regression model with a specific lambda
    ridgereg.fit(X_poly, y_test) # fit the model on the polynomial features
    
    
    #Computing the rss,mse,rmse,r2 score
    y_pred = ridgereg.predict(ridgecoef.fit_transform(X_test))
    rss = np.sum(np.square(y_test - y_pred))
    mse = mean_squared_error(y_test, y_pred)
    rmse = mse**0.5

    print("Polynomial regression with degree "+str(degree) + " and lambda = " + str(i))
    print("r2 score = " + str(r2_score(y_test, y_pred))) 
    print("rss = " + str(rss))
    print("mse = " + str(rmse))
    print("rmse = " + str(rmse))
    print(ridgereg.coef_) # model coefficients

#### 9.4 Lasso Regression

##### 9.4.1 On Train Set

In [ ]:
# Applying Lasso Regression with varying the hyperparameter 'lambda'on Train Set

lambdas = [0.001, 0.01, 0.1, 1, 10, 100, 1000]
for i in lambdas:
    degree = 2
    # Creating degree 5 features
    lassocoef = PolynomialFeatures(degree)
    # Transforming input features to polynomial features (1, x1, x2)
    X_poly = lassocoef.fit_transform(X_train)
    lassoreg = Lasso(alpha = i)
    lassoreg.fit(X_poly, y_train)
 

    # #Computing the rss,mse,rmse,r2 score
    y_pred = lassoreg.predict(lassocoef.fit_transform(X_train))
    rss = np.sum(np.square(y_train - y_pred))
    mse = mean_squared_error(y_train, y_pred)
    rmse = mse**0.5

    
    print("Polynomial regression with degree "+str(degree) + " and lambda = " + str(i))
    print("r2 score = " + str(r2_score(y_train, y_pred))) 
    print("rss = " + str(rss))
    print("mse = " + str(rmse))
    print("rmse = " + str(rmse))
    print(lassoreg.coef_)


##### 9.4.2 On Test Set

In [ ]:
# Applying Lasso Regression with varying the hyperparameter 'lambda' on Train

lambdas = [0.001, 0.01, 0.1, 1, 10, 100, 1000]
for i in lambdas:
    degree = 2
    # Creating degree 5 features
    lassocoef = PolynomialFeatures(degree)
    # Transforming input features to polynomial features (1, x1, x2)
    X_poly = lassocoef.fit_transform(X_test)
    lassoreg = Lasso(alpha = i)
    lassoreg.fit(X_poly, y_test)


    # Compute R^2 
    y_pred = lassoreg.predict(lassocoef.fit_transform(X_test))
    rss = np.sum(np.square(y_test - y_pred))
    mse = mean_squared_error(y_test, y_pred)
    rmse = mse**0.5

    
    print("Polynomial regression with degree "+str(degree) + " and lambda = " + str(i))
    print("r2 score = " + str(r2_score(y_test, y_pred))) 
    print("rss = " + str(rss))
    print("mse = " + str(rmse))
    print("rmse = " + str(rmse))
    print(lassoreg.coef_)

In [ ]:
output.to_csv('New Popularity Prediction.csv',index=False)